In [32]:
import os
from dotenv import load_dotenv
import cv2
import numpy as np
import json

load_dotenv()

True

In [33]:
CURRENT_PATH = os.getenv('CURRENT_PATH')
with open(CURRENT_PATH + "/factory/context/on_chain_security/materials.json", "r") as f:
        materials = json.load(f)
        print(materials)

{'bottom_text': [{'emotion': 'angry', 'situation': 'rugged', 'text': 'Write down what address, what chain, and report to XYZ what happened'}, {'emotion': 'angry', 'situation': 'drained', 'text': 'Write down what Chain, the address that drained and keep on eye on what projects they touch'}, {'emotion': 'frustration', 'situation': 'limbo', 'text': 'Keep Calm and Build On. Only you can pump your own bags'}, {'emotion': 'shame', 'situation': 'rugged', 'text': 'Write down what address, what chain, and report to XYZ what happened'}, {'emotion': 'shame', 'situation': 'drained', 'text': 'Write down what Chain, the address that drained and keep on eye on what projects they touch'}, {'emotion': 'Happiness', 'situation': 'fund_retrieval_successful', 'text': 'Remember to stay alert and tell others who helped you and how!'}], 'top_text': [{'emotion': 'angry', 'situation': 'rugged', 'text': 'Rugged? Take a deep breath first.'}, {'emotion': 'angry', 'situation': 'drained', 'text': 'Drained? Take a de

In [34]:
from factory import *
from factory_widgets import *

In [35]:
## Backend Interface is 
# 1) Grab emotion and situation
# 2) Search through materials
# 3) Create appropriate combination


In [36]:
emotion, situation = "frustration", "limbo"

pic = emotion_and_situation_to_picture(emotion, situation, materials)
tt = emotion_and_situation_to_top_text(emotion, situation, materials)
bt = emotion_and_situation_to_bottom_text(emotion, situation, materials)

In [37]:
pic

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [38]:
tt, bt

("Why isn't my bag pumping?",
 'Keep Calm and Build On. Only you can pump your own bags')

In [39]:
h, w, d = pic.shape
h, w, d

(512, 512, 3)

In [40]:
color = grab_background_color(pic, "corners_and_edges")
color

(0, 0, 0)

In [41]:
top_pad = create_pad_and_text_wif_font_scaling(
    color, h//3, w, d, 
    tt, (w//2, h//6), "center"
)
bottom_pad = create_pad_and_text_wif_font_scaling(
    color, h//3, w, d, 
    bt, (w//2, h//6), "center"
)


(255, 255, 255)


error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'putText'
> Overload resolution failed:
>  - Scalar value for argument 'color' is not numeric
>  - Scalar value for argument 'color' is not numeric


In [59]:
new_pic = concatenate_images_inline([top_pad, pic, bottom_pad], horizontal=False)
cv2.imwrite("test_propaganda.png", new_pic)

True